<a href="https://colab.research.google.com/github/divaamahajan/Spark-Logs/blob/kafkaLogs/kafkaConsumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up Dependencies

In [1]:
!pip install confluent-kafka==2.1.1
!pip install kafka==1.3.5
!pip install kafka-python==2.0.2
!pip install numpy==1.24.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pkg_resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg_resources==0.0.0


In [2]:
!pip install pkg_resources==0.0.0
!pip install py4j==0.10.9.7
!pip install pyspark==3.4.0
!pip install python-dateutil==2.8.2
!pip install pytz==2023.3
!pip install six==1.16.0
!pip install spark==0.2.1
!pip install tzdata==2023.3
!apt-get install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pkg_resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg_resources==0.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0e31bf55dd18e5d197038ab1f3798d6b324441b87eef08fc72bad76b3d59e85d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes:

In [9]:
!pip install confluent_kafka sseclient kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Using cached kafka-1.3.5-py2.py3-none-any.whl (207 kB)
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5565 sha256=61d6dd5079fa7cf3cf90859aaa5a0175652d97d49dd4f0fa5e4b0cf66c0ff856
  Stored in directory: /root/.cache/pip/wheels/60/57/0e/09b1264923280e935a34cc543b7f147f5df12490bd7a992f42
Successfully built sseclient


In [18]:
!pip uninstall kafka
!pip uninstall kafka-python
!pip install kafka-python

Found existing installation: kafka 1.3.5
Uninstalling kafka-1.3.5:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/kafka-1.3.5.dist-info/*
    /usr/local/lib/python3.10/dist-packages/kafka/*
Proceed (Y/n)? Y
  Successfully uninstalled kafka-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [3]:
import zipfile
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from confluent_kafka import Producer, Consumer, KafkaError

from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import StringType
from pyspark.sql.functions import from_json


## Set up the Kafka consumer configuration and topic

In [13]:
# Specify the Kafka broker and topic to consume from
brokers = 'localhost:9092'
topic = 'log_topic'
conf = {
    'bootstrap.servers': brokers,
    'group.id': 'python-consumer',
    'auto.offset.reset': 'earliest',
    'api.version.request': True
}
parquet_host = 'localhost:9000'


In [14]:
dataset = '42MBSmallServerLog.log'
zip_file_path = f'{dataset}.zip'
extracted_folder_path = 'extracted_dataset'

## 1. Set up the Kafka consumer 
Kafka Consumer with Spark (Python)

In [15]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import regexp_extract
from pyspark import SparkContext
from pyspark.sql.types import StringType
# Import the KafkaConsumer from confluent_kafka
from confluent_kafka import KafkaException, KafkaError
from confluent_kafka import Consumer, KafkaError

# Function to process the log lines RDD
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second


In [16]:

def process_logs(rdd):
    if not rdd.isEmpty():
        # Convert RDD to DataFrame
        logs_df = spark.read.json(rdd)

        # Data wrangling steps
        host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
        ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
        method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
        status_pattern = r'\s(\d{3})\s'
        content_size_pattern = r'\s(\d+)$'

        logs_df = logs_df.withColumn('host', regexp_extract('value', host_pattern, 1))
        logs_df = logs_df.withColumn('timestamp', regexp_extract('value', ts_pattern, 1))
        logs_df = logs_df.withColumn('method', regexp_extract('value', method_uri_protocol_pattern, 1))
        logs_df = logs_df.withColumn('endpoint', regexp_extract('value', method_uri_protocol_pattern, 2))
        logs_df = logs_df.withColumn('protocol', regexp_extract('value', method_uri_protocol_pattern, 3))
        logs_df = logs_df.withColumn('status', regexp_extract('value', status_pattern, 1).cast('integer'))
        logs_df = logs_df.withColumn('content_size', regexp_extract('value', content_size_pattern, 1).cast('integer'))

        # Add columns for year, month, and date
        logs_df = logs_df.withColumn('year', year(logs_df['timestamp']))
        logs_df = logs_df.withColumn('month', month(logs_df['timestamp']))
        logs_df = logs_df.withColumn('date', dayofmonth(logs_df['timestamp']))

        # Add columns for hour, minute, and second
        logs_df = logs_df.withColumn('hour', hour(logs_df['timestamp']))
        logs_df = logs_df.withColumn('minute', minute(logs_df['timestamp']))
        logs_df = logs_df.withColumn('second', second(logs_df['timestamp']))

        # Print the transformed DataFrame
        logs_df.show(10, truncate=True)

        # Store the transformed DataFrame as Parquet files
        logs_df.write.parquet('logs.parquet')

        # Store the Parquet files in HDFS
        # writes the transformed DataFrame to the local filesystem as Parquet files with the name logs.parquet. 
        # These Parquet files are created in the same directory where the consumer.py script is executed.
        logs_df.write.parquet('hdfs://localhost:9000/logs.parquet')

        # Download the processed DataFrame as CSV
        logs_df.write.csv('processed_logs.csv')


In [17]:

def get_kafka_stream(consumer):
    num = 0
    while True:
        try:
            message = consumer.poll(1.0)  # Poll for a single message
            if not message: 
                print("no msg")
                continue
            if message.error():
                print("Consumer error: {}".format(message.error()))
                continue
            num += 1
            print("before",num)
            yield message.value().decode('utf-8')
            print("after",num)

        except KeyboardInterrupt:
            break


In [18]:
# Create a SparkSession
spark = SparkSession.builder.appName('KafkaConsumer').getOrCreate()

# Create a StreamingContext with a batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)

In [19]:
# Set up the Kafka consumer
consumer = Consumer(conf)
print('Kafka Consumer has been initiated...')


Kafka Consumer has been initiated...


### Subscribe to the Kafka topic

In [20]:

# Determine the name of the topic that should be consumed and subscribe to it.
topics = consumer.list_topics().topics
print('Available topics to consume: ', topics)

# Subscribe to the Kafka topic
if topic in topics:
    consumer.subscribe([topic])
    print('Subscribed to topic:', topic)
else:
    print('Topic', topic, 'does not exist.')

Available topics to consume:  {'log_lines_topic': TopicMetadata(log_lines_topic, 1 partitions), 'log_topic': TopicMetadata(log_topic, 1 partitions), '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}
Subscribed to topic: log_topic


In [27]:
# Create a streaming DataFrame from Kafka
df = (spark
      .readStream
      .format('kafka')
      .option('kafka.bootstrap.servers', brokers)
      .option('subscribe', topic)
      .option('startingOffsets', 'earliest')
      .load())

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [ ]:
from pyspark.sql.types import StringType

# Convert binary to string key and value
df1 = (df
    .withColumn("key", df["key"].cast(StringType()))
    .withColumn("value", df["value"].cast(StringType())))

In [ ]:
kafka_stream = ssc.queueStream([get_kafka_stream()])

In [ ]:

# Get the log lines from the Kafka stream
lines = kafka_stream.map(lambda x: x)

### convert the log lines to DataFrame and apply SQL queries

In [ ]:

# Function to process the log lines RDD
def process_logs(rdd):
    if not rdd.isEmpty():
        # Convert RDD to DataFrame
        logs_df = spark.read.json(rdd)

        # Perform required transformations and exploratory data analysis
        # For example, you can apply SQL queries, aggregations, or filters on the DataFrame
        transformed_df = logs_df.select('host', 'timestamp', 'method', 'endpoint', 'protocol', 'status', 'content_size')

        # Print the transformed DataFrame
        transformed_df.show()

        # Store the transformed DataFrame as Parquet files
        transformed_df.write.parquet('logs.parquet')

        # Store the Parquet files in HDFS
        transformed_df.write.parquet(f'hdfs://{parquet_host}/logs.parquet')

lines.foreachRDD(lambda rdd: process_logs(rdd))

### Start the streaming context

In [ ]:
ssc.start()
ssc.awaitTermination()

## 4. Parquet File Creation (Python)

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName('ParquetFileCreation').getOrCreate()

# Read the log data file into a DataFrame
logs_rdd = spark.sparkContext.textFile(f'{dataset}')
logs_df = spark.createDataFrame(logs_rdd, "string").toDF("log_line")

# Perform required transformations and analysis on the log data
# For example, you can extract structured attributes using regular expressions
transformed_df = process_logs(logs_df)

# Store the transformed DataFrame as Parquet files
transformed_df.write.parquet('logs.parquet')


## 5. Storing Parquet Files in HDFS (Python)

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName('ParquetFileHDFS').getOrCreate()

# Read the Parquet file into a DataFrame
logs_df = spark.read.parquet('logs.parquet')

# Store the Parquet files in HDFS
logs_df.write.parquet(f'hdfs://{parquet_host}/logs.parquet')
